<a href="https://colab.research.google.com/github/Purkott/Matriz-de-distancias/blob/main/Matriz_de_dist%C3%A2ncias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sua tarefa será gerar uma matriz de distância, computando o cosseno do ângulo entre todos os 
vetores que encontramos usando o tf-idf. Para isso use a seguinte fórmula para o cálculo do cosseno 
use  a  fórmula  apresentada  em  Word2Vector  (frankalcantara.com) 
(https://frankalcantara.com/Aulas/Nlp/out/Aula4.html#/0/4/2) e apresentada na figura a seguir:  
 
O resultado  deste trabalho  será uma matriz que relaciona cada um dos vetores já calculados 
com todos os outros vetores disponíveis na matriz termo-documento.  
 

In [ ]:
from os import X_OK
#Aluno: Fernando Purkott
from bs4 import BeautifulSoup
import requests
import spacy
from spacy.language import Language
import re
import string
import nltk
from nltk.corpus import stopwords
import numpy as np
from scipy import spatial



nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


@Language.component("Space")
def set_custom_boundaries(doc):
     for token in doc[:-1]:
         if token.text.__contains__('  '):
             doc[token.i+1].is_sent_start = True
     return doc



nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('Space', before="parser")
def excluiNovaLinha(value):
    return ''.join(value.splitlines())

def Clean_Text(text):
    text_sem_num = re.sub(r'\d+', '', text)
    text_sem_pontuacao = "".join([char.lower() for char in text_sem_num if char not in string.punctuation]) 
    text_no_doublespace = re.sub('\s+', ' ', text_sem_pontuacao).strip()
    return text_no_doublespace

def remove_stopwords(sentence):
    tokens = sentence.split(" ")
    tokens_filtered = []
    for i in tokens:
      if i in stop_words:
        tokens_filtered.append("")
      else:
        tokens_filtered.append(i)
      for i in range(len(tokens_filtered)):
        if tokens_filtered[i].isspace() or tokens_filtered[i] == "":
          tokens_filtered.pop(i)
    return (" ").join(tokens_filtered)

def scrape(URL):
  blacklist = [
    '[document]',
    'noscript',
    'header',
    'html',
    'meta',
    'head', 
    'input',
    'script',
    'style'
  ]

  req = requests.get(URL)
  soup = BeautifulSoup(req.content)


  txt = soup.find_all(text=True)
  clean_txt = ""

  for t in txt:
    if t.parent.name not in blacklist:
      t = t.lower()
      t = remove_stopwords(t)
      t = Clean_Text(t)
      clean_txt += '{} '.format(t)

  txt_doc = nlp(excluiNovaLinha(clean_txt))

  sentences = list(txt_doc.sents)
  return sentences

def criaVocab(sentences):
  vocab = []
  sentenceWords = []
  for s in sentences:
    for i in s:
      sentenceWords = str(i).split()
      for j in sentenceWords:
        j = j.lower()
        if j.translate(str.maketrans('', '', string.punctuation)) != "":
          vocab.append(j.translate(str.maketrans('', '', string.punctuation)))  
      sentenceWords = []
    vocab = unico(vocab)
    return vocab
  

def unico(words):
  seen = set()
  return [x for x in words if not (x in seen or seen.add(x))]

def vectorize(tokens, vocab):
    vector=[]
    for w in vocab:
      x = 0
      for i in tokens:
        if i == w:
          x +=1
      vector.append(x)
    return vector

def BagOfWords(sentences):
  zeros = []
  vocab = criaVocab(sentences)
  for i in range(len(vocab)):
    zeros.append(0)
  numSentences = 0
  
  for i in sentences:
    for j in i:
      numSentences = numSentences + 1
  bagofwords = []
  clean_sentences = []
  bagofwords.append(vocab)
  currentSentence = 0
  for i in sentences:
    for j in i:
      tokens = str(j).split()
      for z in range(len(tokens)):
        tokens[z] = tokens[z].lower()  
      vector = vectorize(tokens, vocab)
      if vector == zeros:
        i.pop(i.index(j))
      else:
        bagofwords.append(vector)
        clean_sentences.append(j)
  return clean_sentences, bagofwords
      
def TF(sentences, bagofwords):
  zeros = []
  for i in range(len(bagofwords[0])):
    zeros.append(0)
  tf = []
  for i in range(len(bagofwords)-1):
    tf.append(zeros.copy())
  currentSentence = 0
  
  for sentence in clean_sentences:   
    numberofwords = len(str(sentence).split())
    for word in str(sentence).split():
      if word.lower() in bagofwords[0]:
        tf[currentSentence][bagofwords[0].index(word.lower())] = bagofwords[currentSentence+1][bagofwords[0].index(word.lower())] / numberofwords
       
    currentSentence = currentSentence +1
  return tf

def IDF(bagofwords):
  zeros = []
  for i in range(len(bagofwords[0])):
    zeros.append(0)
  idf = zeros
  currentSentence = 0
  for w in bagofwords[0]:
    x = 0      
    for i in bagofwords[1:]:
      if i[bagofwords[0].index(w)] > 0: 
        x += 1
    try:     
      idf[bagofwords[0].index(w)] =  np.log10(len(bagofwords[1:]) / x)
    except:
       idf[bagofwords[0].index(w)] = 1
  return idf

def TFIDF(tf,idf):
  zeros = []
  for i in range(len(tf[0])):
    zeros.append(0)
  tfidf = []
  for i in range(len(tf)):
    tfidf.append(zeros.copy())

  for i in range(len(tfidf)):
    for j in range(len(tfidf[0])):
      tfidf[i][j] = tf[i][j] * idf[j] 
  return tfidf

def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a) * np.linalg.norm(b))
    return cos_sim

def matrizDistancia(tfidf):
  zeros = []
  for i in range(len(tfidf)):
    zeros.append(0)
  md = []
  for i in range(len(tfidf)):
    md.append(zeros.copy())
  for i in range(len(tfidf)):
    for j in range(len(tfidf)):
      md[i][j] = cosine_sim(tfidf[i], tfidf[j])    
  return md
sentences1 = scrape("https://www.ibm.com/cloud/learn/natural-language-processing") 
sentences2 = scrape("https://hbr.org/2022/04/the-power-of-natural-language-processing")
sentences3 = scrape("https://link.springer.com/article/10.1007/s11042-022-13428-4")
sentences4 = scrape("https://www.techtarget.com/searchenterpriseai/definition/natural-language-processing-NLP")
sentences5 = scrape("https://monkeylearn.com/natural-language-processing/")

sentences = [sentences1, sentences2, sentences3, sentences4, sentences5]

clean_sentences, bagofwords = BagOfWords(sentences)
tf = TF(clean_sentences, bagofwords)
idf = IDF(bagofwords)
tfidf = TFIDF(tf,idf)
md = matrizDistancia(tfidf)
for i in md:
  print(i)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[1.0, 0.0, 0.0, 0.36744770212614586, 0.4923742113059574, 0.29632109728851924, 0.2916029457021356, 0.3677486591443577, 0.0, 0.0, 0.4923742113059574, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7117571216231888, 0.4203360917417764, 0.0, 0.39910428479775206, 0.05545271301694889, 0.060304965335089625, 0.017430347442778743, 0.06543340265720694, 0.01048598957776723, 0.0, 0.10690848473256745, 0.09585626702056453, 0.5552688993944049, 0.0, 0.4245392138991577, 0.44607748179109613, 0.0, 0.49237421130595743, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.49237421130595743, 0.0, 0.0, 0.0964544168400251, 0.011664929518612992, 0.0, 0.0, 0.021403963653396692, 0.08562670697039428, 0.016450995111174108, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.33254865642800946, 0.0, 0.0, 0.0, 0.0, 0.49237421130595743, 0.19651534485306021, 0.0, 0.0503635373540254, 0.08462158158301898, 0.0, 0.0, 0.0, 0.11609870183675954, 0.027741432187124136, 0.0, 0.0, 0.03691477623811544, 0.0, 0.03259813585923449